In [ ]:
# example of preparing the horses and zebra dataset
from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
 
# load all images in a directory into memory
def load_images(path, size=(256,256)):
	data_list = list()
	# enumerate filenames in directory, assume all are images
	for filename in listdir(path):
		# load and resize the image
		pixels = load_img(path + filename, target_size=size)
		# convert to numpy array
		pixels = img_to_array(pixels)
		# store
		data_list.append(pixels)
	return asarray(data_list)
 
# dataset path
path = 'maps/'
# load dataset A
dataA1 = load_images(path + 'trainA/')
dataAB = load_images(path + 'testA/')
dataA = vstack((dataA1, dataAB))
print('Loaded dataA: ', dataA.shape)
# load dataset B
dataB1 = load_images(path + 'trainB/')
dataB2 = load_images(path + 'testB/')
dataB = vstack((dataB1, dataB2))
print('Loaded dataB: ', dataB.shape)
# save as compressed numpy array
filename = 'maps_cycleGAN_256.npz'
savez_compressed(filename, dataA, dataB)
print('Saved dataset: ', filename)

In [ ]:
# example of training a cyclegan on the horse2zebra dataset
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from matplotlib import pyplot

# define the discriminator model
def define_discriminator(image_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_image = Input(shape=image_shape)
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    # define model
    model = Model(in_image, patch_out)
    # compile model
    model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
    return model

# generator a resnet block
def resnet_block(n_filters, input_layer):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # first layer convolutional layer
    g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # second convolutional layer
    g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    # concatenate merge channel-wise with input layer
    g = Concatenate()([g, input_layer])
    return g

# define the standalone generator model
def define_generator(image_shape, n_resnet=9):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=image_shape)
    # c7s1-64
    g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # d128
    g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # d256
    g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # R256
    for _ in range(n_resnet):
        g = resnet_block(256, g)
    # u128
    g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # u64
    g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    # c7s1-3
    g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    out_image = Activation('tanh')(g)
    # define model
    model = Model(in_image, out_image)
    return model

# define a composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
    # ensure the model we're updating is trainable
    g_model_1.trainable = True
    # mark discriminator as not trainable
    d_model.trainable = False
    # mark other generator model as not trainable
    g_model_2.trainable = False
    # discriminator element
    input_gen = Input(shape=image_shape)
    gen1_out = g_model_1(input_gen)
    output_d = d_model(gen1_out)
    # identity element
    input_id = Input(shape=image_shape)
    output_id = g_model_1(input_id)
    # forward cycle
    output_f = g_model_2(gen1_out)
    # backward cycle
    gen2_out = g_model_2(input_id)
    output_b = g_model_1(gen2_out)
    # define model graph
    model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
    # define optimization algorithm configuration
    opt = Adam(lr=0.0002, beta_1=0.5)
    # compile model with weighting of least squares loss and L1 loss
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
    return model

# load and prepare training images
def load_real_samples(filename):
    # load the dataset
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]

# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return X, y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
    # generate fake instance
    X = g_model.predict(dataset)
    # create 'fake' class labels (0)
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

# save the generator models to file
def save_models(step, g_model_AtoB, g_model_BtoA):
    # save the first generator model
    filename1 = 'g_model_AtoB_%06d.h5' % (step+1)
    g_model_AtoB.save(filename1)
    # save the second generator model
    filename2 = 'g_model_BtoA_%06d.h5' % (step+1)
    g_model_BtoA.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, trainX, name, n_samples=5):
    # select a sample of input images
    X_in, _ = generate_real_samples(trainX, n_samples, 0)
    # generate translated images
    X_out, _ = generate_fake_samples(g_model, X_in, 0)
    # scale all pixels from [-1,1] to [0,1]
    X_in = (X_in + 1) / 2.0
    X_out = (X_out + 1) / 2.0
    # plot real images
    for i in range(n_samples):
        pyplot.subplot(2, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X_in[i])
    # plot translated image
    for i in range(n_samples):
        pyplot.subplot(2, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(X_out[i])
    # save plot to file
    filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
    pyplot.savefig(filename1)
    pyplot.close()

# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
    selected = list()
    for image in images:
        if len(pool) < max_size:
            # stock the pool
            pool.append(image)
            selected.append(image)
        elif random() < 0.5:
            # use image, but don't add it to the pool
            selected.append(image)
        else:
            # replace an existing image and use replaced image
            ix = randint(0, len(pool))
            selected.append(pool[ix])
            pool[ix] = image
    return asarray(selected)

# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
    # define properties of the training run
    n_epochs, n_batch, = 50, 1
    # determine the output square shape of the discriminator
    n_patch = d_model_A.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    # prepare image pool for fakes
    poolA, poolB = list(), list()
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print(n_steps)
    # manually enumerate epochs
    for i in range(n_steps):
        import time
        T1 = time.time()
        # select a batch of real samples
        X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
        X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
        X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
        # update fakes from pool
        X_fakeA = update_image_pool(poolA, X_fakeA)
        X_fakeB = update_image_pool(poolB, X_fakeB)
        # update generator B->A via adversarial and cycle loss
        g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
        # update discriminator for A -> [real/fake]
        dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
        dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
        # update generator A->B via adversarial and cycle loss
        g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
        # update discriminator for B -> [real/fake]
        dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
        dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
        # summarize performance
        print('Step %d took %s seconds, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1,time.time()-T1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
        # evaluate the model performance every so often
        if (i+1) % 100 == 0:
            # plot A->B translation
            summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
            # plot B->A translation
            summarize_performance(i, g_model_BtoA, trainB, 'BtoA')
            # save the models
            save_models(i, g_model_AtoB, g_model_BtoA)

# load image data
dataset = load_real_samples('maps_cycleGAN_256.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)
# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)

Loaded (2194, 256, 256, 3) (2194, 256, 256, 3)
109700
Step 1 took 87.44078373908997 seconds, dA[0.646,1.043] dB[1.032,0.894] g[21.288,20.048]
Step 2 took 34.53094840049744 seconds, dA[0.901,1.098] dB[2.129,1.100] g[20.595,20.737]
Step 3 took 40.792741775512695 seconds, dA[2.479,1.025] dB[2.697,1.274] g[21.055,20.368]
Step 4 took 42.02374267578125 seconds, dA[2.059,2.355] dB[2.164,1.708] g[20.811,20.753]
Step 5 took 36.45275044441223 seconds, dA[0.865,2.810] dB[1.477,0.905] g[18.970,20.097]
Step 6 took 35.56735372543335 seconds, dA[1.338,3.049] dB[1.370,0.803] g[19.999,21.543]
Step 7 took 36.05255651473999 seconds, dA[0.960,5.231] dB[1.224,1.354] g[20.192,22.406]
Step 8 took 36.89954233169556 seconds, dA[0.950,5.256] dB[1.404,3.177] g[20.740,22.943]
Step 9 took 36.20101761817932 seconds, dA[0.750,4.549] dB[2.013,1.968] g[21.252,21.614]
Step 10 took 36.89896082878113 seconds, dA[0.661,3.563] dB[1.106,1.601] g[18.930,19.209]
Step 11 took 34.02974987030029 seconds, dA[0.611,1.838] dB[0.765

Step 93 took 31.847293853759766 seconds, dA[0.191,0.244] dB[0.062,0.194] g[14.287,12.371]
Step 94 took 30.430177450180054 seconds, dA[0.251,0.287] dB[0.079,0.161] g[14.470,12.747]
Step 95 took 30.610000133514404 seconds, dA[0.248,0.298] dB[0.099,0.122] g[14.526,12.916]
Step 96 took 30.509997606277466 seconds, dA[0.178,0.153] dB[0.215,0.125] g[13.118,12.467]
Step 97 took 30.416001558303833 seconds, dA[0.220,0.285] dB[0.111,0.197] g[11.630,11.065]
Step 98 took 30.343268632888794 seconds, dA[0.221,0.277] dB[0.248,0.056] g[13.652,11.712]
Step 99 took 30.22508215904236 seconds, dA[0.228,0.190] dB[0.098,0.118] g[13.226,11.318]
Step 100 took 30.386000871658325 seconds, dA[0.140,0.417] dB[0.082,0.093] g[15.065,14.292]
>Saved: g_model_AtoB_000100.h5 and g_model_BtoA_000100.h5
Step 101 took 30.18367338180542 seconds, dA[0.299,0.165] dB[0.066,0.071] g[13.628,12.064]
Step 102 took 30.999128818511963 seconds, dA[0.149,0.105] dB[0.109,0.097] g[11.057,9.674]
Step 103 took 30.560001373291016 seconds, 

Step 183 took 32.33634543418884 seconds, dA[0.039,0.026] dB[0.044,0.107] g[12.061,11.953]
Step 184 took 32.33200001716614 seconds, dA[0.105,0.038] dB[0.039,0.049] g[11.263,11.556]
Step 185 took 31.116063833236694 seconds, dA[0.093,0.077] dB[0.060,0.276] g[9.747,9.815]
Step 186 took 31.09609055519104 seconds, dA[0.200,0.428] dB[0.063,0.125] g[15.482,14.705]
Step 187 took 32.00237488746643 seconds, dA[0.195,0.064] dB[0.045,0.056] g[15.971,14.999]
Step 188 took 34.570515871047974 seconds, dA[0.083,0.071] dB[0.033,0.132] g[15.159,14.597]
Step 189 took 35.29652500152588 seconds, dA[0.120,0.159] dB[0.054,0.126] g[13.944,11.690]
Step 190 took 34.80217003822327 seconds, dA[0.134,0.110] dB[0.032,0.079] g[14.062,12.066]
Step 191 took 33.518611669540405 seconds, dA[0.085,0.027] dB[0.033,0.075] g[14.257,13.254]
Step 192 took 31.72300434112549 seconds, dA[0.190,0.074] dB[0.032,0.060] g[13.425,11.560]
Step 193 took 31.29858374595642 seconds, dA[0.051,0.240] dB[0.040,0.387] g[11.198,11.196]
Step 194 

Step 274 took 30.726999759674072 seconds, dA[0.040,0.020] dB[0.036,0.064] g[13.934,12.853]
Step 275 took 30.319092988967896 seconds, dA[0.054,0.013] dB[0.053,0.037] g[13.795,12.615]
Step 276 took 30.567384958267212 seconds, dA[0.162,0.105] dB[0.120,0.283] g[9.982,9.317]
Step 277 took 30.694000482559204 seconds, dA[0.092,0.031] dB[0.039,0.022] g[12.434,10.874]
Step 278 took 30.812000274658203 seconds, dA[0.055,0.088] dB[0.037,0.084] g[12.614,10.864]
Step 279 took 30.997344493865967 seconds, dA[0.110,0.027] dB[0.037,0.103] g[11.294,10.440]
Step 280 took 30.280651569366455 seconds, dA[0.038,0.073] dB[0.028,0.054] g[11.047,10.199]
Step 281 took 30.109525203704834 seconds, dA[0.195,0.041] dB[0.030,0.048] g[12.213,10.368]
Step 282 took 30.378999948501587 seconds, dA[0.063,0.037] dB[0.065,0.111] g[10.116,9.627]
Step 283 took 30.749247550964355 seconds, dA[0.049,0.058] dB[0.026,0.045] g[11.151,10.051]
Step 284 took 30.778000831604004 seconds, dA[0.042,0.044] dB[0.023,0.051] g[13.708,12.701]
St

Step 364 took 30.836352825164795 seconds, dA[0.051,0.097] dB[0.016,0.037] g[12.382,10.482]
Step 365 took 30.862327337265015 seconds, dA[0.065,0.028] dB[0.024,0.087] g[12.099,10.915]
Step 366 took 31.378682613372803 seconds, dA[0.023,0.028] dB[0.044,0.046] g[13.512,11.849]
Step 367 took 31.504424810409546 seconds, dA[0.017,0.047] dB[0.032,0.015] g[11.752,10.561]
Step 368 took 30.802034378051758 seconds, dA[0.103,0.038] dB[0.048,0.028] g[9.605,9.788]
Step 369 took 30.818028450012207 seconds, dA[0.108,0.050] dB[0.023,0.022] g[11.254,10.281]
Step 370 took 30.944559574127197 seconds, dA[0.048,0.042] dB[0.014,0.090] g[11.638,10.215]
Step 371 took 30.63314652442932 seconds, dA[0.059,0.129] dB[0.012,0.060] g[10.103,9.196]
Step 372 took 30.7622389793396 seconds, dA[0.088,0.087] dB[0.015,0.030] g[10.305,9.207]
Step 373 took 30.711000204086304 seconds, dA[0.204,0.124] dB[0.017,0.047] g[11.853,10.708]
Step 374 took 31.044649362564087 seconds, dA[0.075,0.102] dB[0.027,0.045] g[12.380,10.632]
Step 3

Step 455 took 32.05715274810791 seconds, dA[0.034,0.032] dB[0.041,0.039] g[12.089,10.933]
Step 456 took 31.22704815864563 seconds, dA[0.055,0.012] dB[0.013,0.071] g[12.625,11.482]
Step 457 took 34.145774126052856 seconds, dA[0.027,0.042] dB[0.031,0.086] g[11.164,10.651]
Step 458 took 34.434972524642944 seconds, dA[0.030,0.075] dB[0.033,0.055] g[12.103,10.527]
Step 459 took 32.22566342353821 seconds, dA[0.022,0.030] dB[0.011,0.074] g[11.578,10.378]
Step 460 took 31.25399947166443 seconds, dA[0.057,0.025] dB[0.081,0.043] g[11.074,10.278]
Step 461 took 31.31400156021118 seconds, dA[0.055,0.044] dB[0.034,0.058] g[8.808,8.617]
Step 462 took 31.258998155593872 seconds, dA[0.025,0.045] dB[0.017,0.042] g[11.908,10.651]
Step 463 took 31.145655155181885 seconds, dA[0.033,0.016] dB[0.027,0.049] g[9.247,10.016]
Step 464 took 31.191694259643555 seconds, dA[0.151,0.036] dB[0.025,0.038] g[12.114,10.360]
Step 465 took 32.55255389213562 seconds, dA[0.077,0.030] dB[0.026,0.029] g[11.749,11.531]
Step 466

Step 546 took 30.52699899673462 seconds, dA[0.021,0.045] dB[0.015,0.021] g[10.954,9.297]
Step 547 took 30.562999486923218 seconds, dA[0.096,0.011] dB[0.036,0.022] g[9.723,9.125]
Step 548 took 30.557188987731934 seconds, dA[0.011,0.030] dB[0.017,0.019] g[11.327,9.974]
Step 549 took 30.91476845741272 seconds, dA[0.049,0.175] dB[0.010,0.023] g[11.444,9.488]
Step 550 took 32.292157888412476 seconds, dA[0.039,0.070] dB[0.007,0.016] g[12.335,11.595]
Step 551 took 32.074384450912476 seconds, dA[0.219,0.060] dB[0.008,0.019] g[10.766,10.211]
Step 552 took 31.818267345428467 seconds, dA[0.165,0.067] dB[0.011,0.024] g[11.960,10.480]
Step 553 took 31.771782636642456 seconds, dA[0.026,0.015] dB[0.013,0.017] g[10.500,9.018]
Step 554 took 31.95163607597351 seconds, dA[0.016,0.014] dB[0.006,0.026] g[11.225,9.677]
Step 555 took 32.311567544937134 seconds, dA[0.017,0.006] dB[0.020,0.020] g[10.570,9.425]
Step 556 took 31.85101056098938 seconds, dA[0.008,0.094] dB[0.005,0.009] g[13.274,12.475]
Step 557 to

Step 637 took 30.52325963973999 seconds, dA[0.122,0.019] dB[0.003,0.006] g[10.014,8.604]
Step 638 took 30.679000854492188 seconds, dA[0.027,0.022] dB[0.004,0.026] g[10.771,9.030]
Step 639 took 30.480997800827026 seconds, dA[0.021,0.138] dB[0.003,0.012] g[10.777,8.654]
Step 640 took 30.589999675750732 seconds, dA[0.185,0.029] dB[0.009,0.020] g[9.441,8.688]
Step 641 took 30.760299921035767 seconds, dA[0.025,0.015] dB[0.007,0.011] g[11.068,9.640]
Step 642 took 30.618001699447632 seconds, dA[0.032,0.116] dB[0.006,0.007] g[11.047,9.061]
Step 643 took 30.457000255584717 seconds, dA[0.017,0.010] dB[0.012,0.007] g[12.610,12.463]
Step 644 took 30.791000604629517 seconds, dA[0.023,0.015] dB[0.007,0.015] g[11.255,10.867]
Step 645 took 30.690470933914185 seconds, dA[0.023,0.014] dB[0.013,0.016] g[9.797,9.352]
Step 646 took 30.555053234100342 seconds, dA[0.017,0.048] dB[0.007,0.013] g[9.717,8.435]
Step 647 took 30.449998378753662 seconds, dA[0.032,0.014] dB[0.009,0.008] g[9.006,8.703]
Step 648 took